# タグ推薦のコードへ渡すためのデータセット構築
- タグ共起ペアを元に，各ハッシュタグに id (通し番号)を付与する。この id が edge_list で利用される。
- タグ共起ペア (src, dst) を 「id のペア」に置き換える。
- 各タグの特徴ベクトルとして one-hot 表現を割り当てる。

In [13]:
# ハッシュタグの共起ペアを読み込む
import pickle
with open("kari.pkl", "rb") as f:
    df = pickle.load(f)
    
df

,src,dst
0,#デリヘル,#保証
1,#symphonia,#在宅ワーク
2,#マスク,#下町
3,#ネガティブ,#自己肯定
4,#簡単FX,#在宅ワーク
...,...,...
2708,#熱中症に気をつけて,#熱中症
2709,#チャトレ,#三ノ宮
2710,#ホテヘル,#東京
2711,#新宿,#新橋


In [14]:
# 上記のデータフレームからタグの一覧を得る
hashtags = list(set(df['src'].tolist() + df['dst'].tolist()))
# tags

# タグと id (連番) を紐付ける
hashtag2id = dict()
id2hashtag = dict() # GNN にて利用する
count = 0
for tag in hashtags:
    hashtag2id[tag] = count
    id2hashtag[count] = tag
    count += 1

#tag2id

# 予測結果を分かりやすくするため，タグ - id の対応表を保存
import pickle

with open("hashtag2id.pkl", "wb") as f:
    pickle.dump(hashtag2id, f)
    
with open("id2hashtag.pkl", "wb") as f:
    pickle.dump(id2hashtag, f)

# データフレームに対して id の対応付けを加える
df['src_id'] = df['src'].apply(lambda x: hashtag2id[x])
df['dst_id'] = df['dst'].apply(lambda x: hashtag2id[x])

df

,src,dst,src_id,dst_id
0,#デリヘル,#保証,456,758
1,#symphonia,#在宅ワーク,821,317
2,#マスク,#下町,635,664
3,#ネガティブ,#自己肯定,50,368
4,#簡単FX,#在宅ワーク,209,317
...,...,...,...,...
2708,#熱中症に気をつけて,#熱中症,259,342
2709,#チャトレ,#三ノ宮,442,814
2710,#ホテヘル,#東京,761,324
2711,#新宿,#新橋,805,385


In [15]:
# edge index を作る
import numpy as np
edge_index = np.vstack([df['src_id'].values, df['dst_id']])
edge_index

array([[456, 821, 635, ..., 761, 805, 598],
       [758, 317, 664, ..., 324, 385, 578]])

In [16]:
# 各タグ（ノード）の特徴ベクトルを one-hot vector として与える
import numpy as np
tag_size = len(hashtags)
features = np.zeros((tag_size, tag_size))
for idx in range(len(hashtags)):
    features[idx][idx] = 1.0

features

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [17]:
# 特徴ベクトルと edge list を pickle にて保存
import pickle
dataset = {"x":features, "edge_index":edge_index}
with open("kari_dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)